In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

In [2]:
file_path = "C:\\Users\\prami\\Desktop\\SCMA 632\\SCMA 632\\assignments\\A6b\\pinksheet.xlsx"
df = pd.read_excel(file_path, sheet_name="Monthly Prices", skiprows=6)

In [3]:
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'].astype(str) + '01', format='%Y%m%d')

ValueError: time data "1960M0101" doesn't match format "%Y%m%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [4]:
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

In [5]:
df['Date'] = pd.to_datetime(df['Date'].astype(str) + '-01', format='%YM%m-%d')

In [6]:
print(df.dtypes)

Date           datetime64[ns]
CRUDE_PETRO           float64
CRUDE_BRENT           float64
CRUDE_DUBAI           float64
CRUDE_WTI              object
                    ...      
NICKEL                float64
Zinc                  float64
GOLD                  float64
PLATINUM              float64
SILVER                float64
Length: 72, dtype: object


In [7]:
commodity = df.iloc[:, [0, 2, 24, 69, 71, 60, 30]].copy()
commodity.columns = commodity.columns.str.lower()

In [8]:
commodity_data = commodity.drop(columns=['date'])

In [9]:
columns_to_test = commodity_data.columns

In [10]:
non_stationary_count = 0
stationary_columns = []
non_stationary_columns = []

In [11]:
for col in columns_to_test:
    adf_result = adfuller(commodity_data[col], regression='c', autolag='AIC')
    p_value = adf_result[1]
    print(f"\nADF test result for column: {col}")
    print(f"ADF Statistic: {adf_result[0]}, p-value: {p_value}")


ADF test result for column: crude_brent
ADF Statistic: -1.5078661910935343, p-value: 0.5296165197702398

ADF test result for column: soybeans
ADF Statistic: -2.42314645274189, p-value: 0.13530977427790403

ADF test result for column: gold
ADF Statistic: 1.3430517021933006, p-value: 0.9968394353612382

ADF test result for column: silver
ADF Statistic: -1.397294710746222, p-value: 0.5835723787985764

ADF test result for column: urea_ee_bulk
ADF Statistic: -2.5101716315209086, p-value: 0.11301903181624645

ADF test result for column: maize
ADF Statistic: -2.4700451060920425, p-value: 0.12293380919376751


In [17]:
if p_value > 0.05:
  non_stationary_count += 1
  non_stationary_columns.append(col)
    else:
        stationary_columns.append(col)

IndentationError: unexpected indent (1888837493.py, line 4)

In [19]:
if p_value > 0.05:
        non_stationary_count += 1
        non_stationary_columns.append(col)
    else:
        stationary_columns.append(col)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [16]:
 # Check if the p-value is greater than 0.05 (commonly used threshold)
    if p_value > 0.05:
        non_stationary_count += 1
        non_stationary_columns.append(col)
    else:
        stationary_columns.append(col)


IndentationError: unexpected indent (2443925374.py, line 2)

In [20]:
print(f"\nNumber of non-stationary columns: {non_stationary_count}")
print(f"Non-stationary columns: {non_stationary_columns}")
print(f"Stationary columns: {stationary_columns}")



Number of non-stationary columns: 0
Non-stationary columns: []
Stationary columns: []


In [21]:
# Co-Integration Test (Johansen's Test)
# Determining the number of lags to use (you can use information criteria like AIC, BIC)
model = VAR(commodity_data)
lags = model.select_order(maxlags=10)
lag_length = lags.selected_orders['aic']  # Choosing the lag with the lowest AIC
